In [21]:
import torch
import onnxruntime
import numpy as np
import cv2

In [12]:
#non max suppression (nms) function
def hard_nms(box_scores, iou_threshold, top_k=-1, candidate_size=200):
    """
    Args:
        box_scores (N, 5): boxes in corner-form and probabilities.
        iou_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
        candidate_size: only consider the candidates with the highest scores.
    Returns:
         picked: a list of indexes of the kept boxes
    """
    scores = box_scores[:, -1]
    boxes = box_scores[:, :-1]
    picked = []
    _, indexes = scores.sort(descending=True)
    indexes = indexes[:candidate_size]
    while len(indexes) > 0:
        current = indexes[0]
        picked.append(current.item())
        if 0 < top_k == len(picked) or len(indexes) == 1:
            break
        current_box = boxes[current, :]
        indexes = indexes[1:]
        rest_boxes = boxes[indexes, :]
        iou = iou_of(
            rest_boxes,
            current_box.unsqueeze(0),
        )
        indexes = indexes[iou <= iou_threshold]

    return box_scores[picked, :]

#utility functions
def iou_of(boxes0, boxes1, eps=1e-5):
    """Return intersection-over-union (Jaccard index) of boxes.
    Args:
        boxes0 (N, 4): ground truth boxes.
        boxes1 (N or 1, 4): predicted boxes.
        eps: a small number to avoid 0 as denominator.
    Returns:
        iou (N): IoU values.
    """
    overlap_left_top = torch.max(boxes0[..., :2], boxes1[..., :2])
    overlap_right_bottom = torch.min(boxes0[..., 2:], boxes1[..., 2:])

    overlap_area = area_of(overlap_left_top, overlap_right_bottom)
    area0 = area_of(boxes0[..., :2], boxes0[..., 2:])
    area1 = area_of(boxes1[..., :2], boxes1[..., 2:])
    return overlap_area / (area0 + area1 - overlap_area + eps)

def area_of(left_top, right_bottom) -> torch.Tensor:
    """Compute the areas of rectangles given two corners.
    Args:
        left_top (N, 2): left top corner.
        right_bottom (N, 2): right bottom corner.
    Returns:
        area (N): return the area.
    """
    hw = torch.clamp(right_bottom - left_top, min=0.0)
    return hw[..., 0] * hw[..., 1]

In [22]:
#constants
image_size = 300
image_mean = np.array([127, 127, 127])  # RGB layout
image_std = 128.0
iou_threshold = 0.45
center_variance = 0.1
size_variance = 0.2

filter_threshold = 0.25
candidate_size=200
sigma=0.5
top_k=20

In [52]:
#loading image and image preprocessing 
image_path = 'SampleFrames_for_Baselineline/frame00448.jpg'

orig_image = cv2.imread(image_path)
image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

image = cv2.resize(image, (image_size, image_size))
image = image.astype(np.float32)
image -= image_mean
image = image/image_std
# image = torch.from_numpy(image.astype(np.float32)).permute(2, 0, 1)
# images = image.unsqueeze(0)
# numpy equivalent of above torch method
image = image.transpose((2, 0, 1))
images = np.expand_dims(image, 0)

# images = images.cpu().numpy()



#loading onnx model and inferencing
ort_session = onnxruntime.InferenceSession("models/baseline_model.onnx",
                                           providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 
                                                      'CPUExecutionProvider'])

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: images}
ort_outs = ort_session.run(None, ort_inputs)

import json
result=[]
for i in range(3000):
    result.append({'conf':ort_outs[0][0][i].tolist(),'bbox':ort_outs[1][0][i].tolist()})
with open('results/frame448_onnx-josh.json','w') as f:
    json.dump(result,f,indent=4)

2022-12-15 15:06:00.596383457 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:509 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


In [53]:
data_422 = ort_outs[0][0].flatten().tolist()
data_471 = ort_outs[1][0].flatten().tolist()
with open('_ns_422_bub.txt', 'w') as f:
    for line in data_422:
        f.write(f"{line}\n")
with open('_ns_471_bub.txt', 'w') as f:
    for line in data_471:
        f.write(f"{line}\n")

In [54]:
#onnx output format
print(ort_outs)

[array([[[9.8295951e-01, 1.3772622e-04, 3.8826594e-04, ...,
         2.9158141e-04, 8.6010905e-04, 1.1549543e-04],
        [9.8894602e-01, 7.9930515e-04, 1.4301903e-03, ...,
         6.5930543e-04, 1.6731346e-03, 3.1344703e-04],
        [9.7481370e-01, 2.2204932e-04, 7.9939316e-04, ...,
         4.8207736e-04, 2.2071155e-03, 2.5556624e-04],
        ...,
        [9.8741257e-01, 3.6753128e-05, 6.4116875e-03, ...,
         1.9436853e-05, 4.1402732e-03, 2.0046044e-05],
        [9.8401690e-01, 3.0069516e-05, 5.6354385e-03, ...,
         2.2533295e-05, 8.5932463e-03, 2.3950246e-05],
        [9.7707158e-01, 4.6546724e-05, 7.4633746e-03, ...,
         2.6527547e-05, 1.2634422e-02, 2.7220520e-05]]], dtype=float32), array([[[ 0.04295783, -0.00497538,  0.07433966,  0.07776529],
        [-0.07389446, -0.08710002,  0.12798768,  0.16818765],
        [ 0.07898747, -0.00732234,  0.11466733,  0.07174507],
        ...,
        [ 0.23108721,  0.15713984,  0.77595186,  0.8258249 ],
        [ 0.15813333,  

In [55]:
#bounding box and scores

scores, boxes = ort_outs
boxes = torch.from_numpy(boxes[0])
scores = torch.from_numpy(scores[0])

print('boxes')
print(boxes.size())
print(boxes)
print('')
print('scores')
print(scores.size())
print(scores)

boxes
torch.Size([3000, 4])
tensor([[ 0.0430, -0.0050,  0.0743,  0.0778],
        [-0.0739, -0.0871,  0.1280,  0.1682],
        [ 0.0790, -0.0073,  0.1147,  0.0717],
        ...,
        [ 0.2311,  0.1571,  0.7760,  0.8258],
        [ 0.1581,  0.2193,  0.8394,  0.6765],
        [ 0.2888,  0.1716,  0.7193,  0.8235]])

scores
torch.Size([3000, 11])
tensor([[9.8296e-01, 1.3773e-04, 3.8827e-04,  ..., 2.9158e-04, 8.6011e-04,
         1.1550e-04],
        [9.8895e-01, 7.9931e-04, 1.4302e-03,  ..., 6.5931e-04, 1.6731e-03,
         3.1345e-04],
        [9.7481e-01, 2.2205e-04, 7.9939e-04,  ..., 4.8208e-04, 2.2071e-03,
         2.5557e-04],
        ...,
        [9.8741e-01, 3.6753e-05, 6.4117e-03,  ..., 1.9437e-05, 4.1403e-03,
         2.0046e-05],
        [9.8402e-01, 3.0070e-05, 5.6354e-03,  ..., 2.2533e-05, 8.5932e-03,
         2.3950e-05],
        [9.7707e-01, 4.6547e-05, 7.4634e-03,  ..., 2.6528e-05, 1.2634e-02,
         2.7221e-05]])


In [56]:
#bounding box decoding and extracting top_k detections above prob_threshold

prob_threshold = filter_threshold
picked_box_probs = []
picked_labels = []
for class_index in range(1, scores.size(1)):
    probs = scores[:, class_index]
    mask = probs > prob_threshold
    probs = probs[mask]
    if probs.size(0) == 0:
        continue
    subset_boxes = boxes[mask, :]
    box_probs = torch.cat([subset_boxes, probs.reshape(-1, 1)], dim=1)
    box_probs = hard_nms(box_probs,iou_threshold, top_k, candidate_size)
    picked_box_probs.append(box_probs)
    picked_labels.extend([class_index] * box_probs.size(0))
if not picked_box_probs:
    boxes, labels, probs = torch.tensor([]), torch.tensor([]), torch.tensor([])
else:
    picked_box_probs = torch.cat(picked_box_probs)
    picked_box_probs[:, 0] *= width
    picked_box_probs[:, 1] *= height
    picked_box_probs[:, 2] *= width
    picked_box_probs[:, 3] *= height
    boxes, labels, probs = picked_box_probs[:, :4], torch.tensor(picked_labels), picked_box_probs[:, 4]

In [57]:
#final bonding box detections, labels and prob/confidence scores

print('bounding boxes')
print(boxes)
print('')
print('labels')
print(labels)
print('')
print('probability/confidence scores')
print(probs)

bounding boxes
tensor([[2654.9373, 1231.6953, 3411.1221, 1679.4271],
        [3645.2175, 1286.9338, 3836.7449, 1368.8589]])

labels
tensor([9, 9])

probability/confidence scores
tensor([0.9968, 0.3093])
